<a href="https://colab.research.google.com/github/dimitriasta/h8-learning-journey/blob/master/h8dsft_P0W3_asta_dimitri.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GRADED CHALLENGE 3

# INTRODUCTION

Hello! Welcome to my Notebook.

Notebook ini merupakan bagian dari tugas *Graded Challenge 3* yang dibuat oleh **Asta Zeska Dimitri** dari Batch **001-HCK**.

## Background

The Look yang merupakan salah satu platform e-commerce terbesar di planet Mars. Sebagai bagian dari tim Data Scientist untuk The Look, kami diminta untuk mengevaluasi performa transaksi dan penjualan pada periode Q1-Q2 tahun 2022. Hasil yang diharapkan adalah ditemukannya masalah yang terjadi dan kemudian dapat ditemukan solusi agar masalah ini tidak terjadi lagi kedepannya.

## The Data

Untuk data yang digunakan kali ini adalah dataset `bigquery-public-data.thelook_ecommerce`. Data ini tersedia di *Google Cloud Platform* dan merupakan bagian dari *BigQuery Public Datasets*.
Untuk tabel yang digunakan adalah tabel orders, order_items, dan events.

Untuk mengkoneksikan notebook ini dengan dataset yang ingin digunakan, digunakan code seperti dibawah ini:

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

from google.cloud import bigquery

project_id = "hacktiv8-asta"
client = bigquery.Client(project=project_id)

Authenticated


# PROBLEMS ANALYSIS

### 1. Berapa persen jumlah user yang retention dari Januari - Juni 2022 dari total keseluruhan user yang ada?

In [2]:
# Nomor 1

client.query('''

SELECT (
  SELECT COUNT(cust_category)
  FROM (
    SELECT user_id AS cust_id, COUNT(order_id) AS total_order,
    CASE WHEN COUNT(order_id) >= 3 THEN 'loyal_customer' ELSE 'regular_customer' END AS cust_category
    FROM `bigquery-public-data.thelook_ecommerce.orders`
    WHERE created_at BETWEEN '2022-01-01 00:00:00 UTC' AND '2022-06-30 23:59:59 UTC'
    GROUP BY user_id HAVING cust_category = 'loyal_customer'))/
       (
    SELECT COUNT(cust_id)
    FROM (
      SELECT user_id AS cust_id, COUNT(order_id) AS total_order
      FROM `bigquery-public-data.thelook_ecommerce.orders`
      WHERE created_at BETWEEN '2022-01-01 00:00:00 UTC' AND '2022-06-30 23:59:59 UTC'
      GROUP BY user_id))
* 100 AS loyal_percentage

''').to_dataframe()

,loyal_percentage
0,3.429814


Untuk Customer Retention kali ini, kami mengklasifikasi customer berdasarkan jumlah transaksinya, dimana untuk transaksi >= 3 akan digolongkan sebagai 'Loyal Customer' sedangkan untuk transaksi < 3 akan digolongkan sebagai 'Regular Customer. Pada periode Q1-Q2 tahun 2022 ini, jumlah Loyal Customer pada platform The Look hanya sekitar 3.4% saja. Tentu nilai ini masih sangat kecil dan tidak sesuai harapan perusahaan. Kecilnya jumlah Loyal Customer The Look dapat mengindikasikan masih kurang bersaingnya kualitas produk yang ditawarkan ataupun belum maksimalnya customer service yang disediakan, sehingga customer masih ragu-ragu untuk melakukan repeat order di platform The Look.


### 2. Berapa rata-rata, minimum, dan maksimum jumlah transaksi yang dilakukan oleh user loyal?

In [3]:
# Nomor 2

client.query('''

SELECT AVG(total_order) AS average_order, MIN(total_order) AS minimum_order, MAX(total_order) AS maximum_order
FROM (
  SELECT user_id AS cust_id, COUNT(order_id) AS total_order,
  CASE WHEN COUNT(order_id) >= 3 THEN 'loyal_customer' ELSE 'regular_customer' END AS cust_category
  FROM `bigquery-public-data.thelook_ecommerce.orders`
  WHERE created_at BETWEEN '2022-01-01 00:00:00 UTC' AND '2022-06-30 23:59:59 UTC'
  GROUP BY user_id HAVING cust_category = 'loyal_customer'
  )

''').to_dataframe()

,average_order,minimum_order,maximum_order
0,3.176916,3,4


Bahkan dapat dilihat bahwa Loyal Customer pada periode Q1-Q2 tahun 2022 ini baru melakukan maksimal 4 transaksi. Dengan nilai rata-ratanya yang lebih dekat dengan nilai minimum, yaitu sekitar 3.18. Terlihat bahwa mayoritas Loyal Customer baru melakukan 3 transaksi saja, hanya sedikit yang melakukan transaksi > 3 kali di platform The Look. Dapat disimpulkan juga bahwa Loyal Customer pun masih merasa ragu-ragu untuk melakukan transaksi di platform The Look.

### 3. Berapa rata-rata, minimum, dan maksimum uang yang dihabiskan oleh user loyal? 

In [4]:
# Nomor 3

client.query('''

SELECT AVG(total_sales) AS average_sales, MIN(total_sales) AS minimum_sales, MAX(total_sales) AS maximum_sales
FROM (
  SELECT cust_id, SUM(order_items.sale_price) AS total_sales
  FROM (
    SELECT user_id AS cust_id, COUNT(order_id) AS total_order,
    CASE WHEN COUNT(order_id) >= 3 THEN 'loyal_customer' ELSE 'regular_customer' END AS cust_category
    FROM `bigquery-public-data.thelook_ecommerce.orders`
    WHERE created_at BETWEEN '2022-01-01 00:00:00 UTC' AND '2022-06-30 23:59:59 UTC'
    GROUP BY user_id HAVING cust_category = 'loyal_customer')
  LEFT JOIN `bigquery-public-data.thelook_ecommerce.order_items` AS order_items
  ON cust_id = order_items.user_id
  GROUP BY cust_id
  )


''').to_dataframe()

,average_sales,minimum_sales,maximum_sales
0,320.044031,40.99,1502.250001


Pada periode Q1-Q2 tahun 2022 ini, persebaran uang yang dibelanjakan Loyal Customer sudah cukup beragam dari 40.99-1502.25 dollars. Dengan rata-ratanya sekitar 320 dollars masih terlihat bahwa persebarannya lebih condong ke nilai minimum. Bisa disimpulkan juga bahwa Loyal Customer belum terlalu berani untuk membelanjakan uang yang lebih banyak di platform The Look.

### 4. Berasal dari sumber trafik mana saja kebanyakan user loyal akses platform The Look?

In [5]:
# Nomor 4

client.query('''

SELECT events.traffic_source, COUNT(events.traffic_source) AS total_traffic
FROM (
  SELECT user_id AS cust_id, COUNT(order_id) AS total_order,
  CASE WHEN COUNT(order_id) >= 3 THEN 'loyal_customer' ELSE 'regular_customer' END AS cust_category
  FROM `bigquery-public-data.thelook_ecommerce.orders`
  WHERE created_at BETWEEN '2022-01-01 00:00:00 UTC' AND '2022-06-30 23:59:59 UTC'
  GROUP BY user_id HAVING cust_category = 'loyal_customer')
LEFT JOIN `bigquery-public-data.thelook_ecommerce.events` AS events
ON cust_id = events.user_id
GROUP BY events.traffic_source

''').to_dataframe()

,traffic_source,total_traffic
0,Adwords,12076
1,Email,18691
2,Facebook,3911
3,YouTube,4171
4,Organic,2109


Dapat dilihat bahwa sumber trafik dari Loyal Customer The Look paling banyak melalui Email lalu disusul oleh Adwords/Google Ads. Lalu platform seperti Facebook dan Youtube tidak terlalu banyak menyumbang trafik ke The Look. Dari data ini dapat terlihat karakteristik/kebiasaan Loyal Customer dari The Look mayoritas masih konservatif, bukan yang terlalu sering melihat social media. Data ini dapat menjadi pertimbangan untuk memilih lewat media/platform apa saja kita ingin menaruh iklan-iklan The Look agar hasilnya lebih efektif.

### 5. Apakah ada hubungan antara sumber trafik dengan loyal atau tidaknya seorang user? 

In [6]:
# Nomor 5

traffic = client.query('''

SELECT events.traffic_source, loyalty
FROM (
  SELECT user_id AS cust_id, COUNT(order_id) AS total_order,
  CASE WHEN COUNT(order_id) >= 3 THEN 'Yes' ELSE 'No' END AS loyalty
  FROM `bigquery-public-data.thelook_ecommerce.orders`
  WHERE created_at BETWEEN '2022-01-01 00:00:00 UTC' AND '2022-06-30 23:59:59 UTC'
  GROUP BY user_id)
LEFT JOIN `bigquery-public-data.thelook_ecommerce.events` AS events
ON cust_id = events.id

''').to_dataframe()

traffic

,traffic_source,loyalty
0,Email,No
1,Email,No
2,Email,No
3,Email,No
4,Email,No
...,...,...
30813,Facebook,No
30814,Facebook,No
30815,Facebook,No
30816,Facebook,No


In [7]:
import pandas as pd
from scipy import stats

**H0 : There is no relation between traffic source and customers loyalty**

**H1 : There is a relation between traffic source and customers loyalty**

In [8]:
contingency_table=pd.crosstab(traffic["traffic_source"], traffic["loyalty"])
contingency_table

loyalty,No,Yes
traffic_source,,
Adwords,8934,309
Email,13415,482
Facebook,2895,112
Organic,1481,65
YouTube,3036,89


In [9]:
stat, p, dof, expected = stats.chi2_contingency(contingency_table)
print('p=%.3f' % (p))
if p > 0.05:
    print('Probably independent')
else:
    print('Probably dependent')

p=0.133
Probably independent


Setelah dilakukan tes hipotesis menggunakan metode chi-square, diketahui bahwa tidak ditemukannya hubungan yang signifikan antara sumber trafik dengan loyalitas customer The Look. Berarti fokus tujuan dari pemasangan iklan di berbagai platform ini adalah untuk menjaring customer-customer baru. Untuk membuat para customer baru ini berubah menjadi Loyal Customer, kita harus melakukan treatment/pelayanan yang tepat agar mereka merasa puas dan nyaman berbelanja di The Look.

# SUCCESS CRITERIA WITH SMART

### **Problem Statement:**

Selama Q1-Q2 tahun 2022, Loyal Customer The Look yang hanya berjumlah 3.4% masih terlihat ragu-ragu untuk melakukan transaksi ataupun membelanjakan uangnya di platform ini. Hal ini mengindikasikan masih kurang bersaingnya kualitas produk yang ditawarkan ataupun belum maksimalnya customer service yang disediakan. The Look juga belum mampu untuk mengubah para customer baru yang datang dari iklan yang dipasang di berbagai media hingga menjadi Loyal Customer.



### **SMART Goals:**

1. Menambah jumlah Loyal Customer The Look sampai 20% pada akhir tahun 2023.
2. Meningkatkan rata-rata transaksi Loyal Customer menjadi 12 transaksi per tahun pada tahun 2024.
3. Meningkatkan rata-rata uang yang dibelanjakan Loyal Customer menjadi minimal 400 dollars pada Q2 2023.
4. Memaksimalkan traffic source untuk menghasilkan paling sedikit 10000 kunjungan ke platform The Look.

### **Plan of Action:**

*   Menyediakan layanan customer service prioritas untuk para Loyal Customer The Look agar segala bentuk komplain bisa ditangani secara cepat dan memuaskan.
*   Membuat/menambah tim Quality Control supaya kondisi produk yang kita kirimkan dalam kondisi yang sempurna, hal ini akan berpengaruh ke berkurangnya komplain dan juga pengembalian pesanan karena kondisi produk yang diterima dalam kondisi rusak/cacat.
*   Memberikan reward kepada Loyal Customer The Look, bisa dalam bentuk special offers, discount codes, tiered rewards, dll.
*   Menambah metode pembayaran yang tersedia di platform, agar memudahkan semua customer The Look dalam proses belanja. Semakin mudah proses belanja di platform maka peluang customer melakukan repeat order akan semakin tinggi.
*   Menentukan channel/media mana yang paling efektif dalam menyampaikan informasi tentang The Look kepada semua customer. Informasi ini bisa berisi program promosi yang sedang berlangsung, produk baru yang ingin dirilis, ataupun pengumuman mengenai fitur baru yang tersedia di platform.








# CONCLUSIONS

Setelah melakukan beberapa analisa untuk platform The Look pada periode Q1-Q2 tahun 2022 ini ditemukan beberapa masalah yang harus ditangani untuk perkembangan perusahaan kedepannya. 

Yang pertama, jumlah Loyal Customer pada platform The Look hanya sekitar 3.4% saja. Kecilnya jumlah Loyal Customer The Look dapat mengindikasikan masih kurang bersaingnya kualitas produk yang ditawarkan ataupun belum maksimalnya customer service yang disediakan, kemudahan dalam proses belanja di platform The Look juga mempengaruhi Loyal Customer untuk melakukan repeat order di platform The Look. 

Selanjutnya, rata-rata transaksi yang dilakukan oleh Loyal Customer hanya 3 kali transaksi saja, dengan rata-rata uang yang dikeluarkan sekitar 320 dollars. Ini juga menjadi indikasi masih adanya keraguan Loyal Customer untuk berbelanja di platform kita.

Kemudian kita menemukan bahwa karakteristik/kebiasaan Loyal Customer dari The Look mayoritas masih konservatif, bukan yang terlalu sering melihat social media. Ini dapat menjadi pertimbangan untuk memilih lewat media/channel apa saja kita ingin menaruh iklan-iklan The Look agar hasilnya lebih efektif.

Dan tujuan dari pemasangan iklan di berbagai media/channel ini adalah untuk menjaring customer-customer baru. Untuk membuat para customer baru ini berubah menjadi Loyal Customer, kita harus melakukan treatment/pelayanan yang tepat agar mereka merasa puas dan nyaman berbelanja di The Look.

# QUESTIONS

1. Apakah problem statement yang kamu definisikan di awal dapat terukur ketercapaiannya? berikan pendapatmu!

  *Jawaban: Bisa, karena problem statement yang didefinisikan diatas adalah hasil yang didapatkan dari analisa menggunakan statistik. Kita mendapatkan nilai-nilai yang kita anggap sebagai indikator sebuah masalah. Ketika selanjutnya kita menentukan goals dan success criteria, kita bisa menggunakan analisa yang sama dan mendapatkan nilai-nilai baru yang nantinya bisa menjadi indikator untuk melakukan evaluasi apakah goals dan success criteria yang kita tentukan berhasil dicapai atau tidak ada perubahan sama sekali.*

2. Apa yang dimaksud dengan customer/user retention?

  *Jawaban: Customer/User Retention adalah suatu metrik/indikator yang menandakan kemampuan suatu perusahaan untuk membuat customernya menggunakan produk/jasanya secara terus menerus, atau dengan kata lain menjadi customer setia/loyal. Dan secara otomaris mengurangi kemungkinan para customer ini menggunakan produk/jasa dari kompetitor.*

3. Jika platform The Look membuat fitur tambahan, apakah kamu akan merekomendasikan/memprioritaskan kepada user loyal? apa pertimbanganmu?

  *Jawaban: Iya, saya akan memprioritaskan customer loyal dalam pengembangan fitur baru/fitur tambahan. Pertama, karena menurut artikel berjudul "21 Surprising Customer Retention Statistics For 2021", kenaikan Customer Retention sebesar 5% saja bisa meningkatkan profit sebesar 25-95%. Ketika customer menjadi setia/loyal ke produk kita, maka akan lebih mudah untuk mereka mencoba produk/jasa baru yang kita tawarkan. Kemudian para customer loyal ini akan mempromosikan produk/servis kita ke orang-orang terdekatnya, membawa customer baru untuk menggunakan produk/jasa kita. Yang kedua, proses customer acquisition membutuhkan biaya yang jauh lebih besar, tapi kedua proses ini memang sama pentingnya untuk keberlangsungan perusahaan.*